<br> Ayman FAHSI | A20440820

Mouhammad BAZZI | A20522180


CS577 - Fall 2022</br> <h1><br><b><font color='red'>Project</font></br></h1>

#### *Credit for the DataBase creators*

**The following is a credit that the creator of the database that we use require us to put in our code in order to use theire database:**


@article

{Rothe-IJCV-2018,

  author = {Rasmus Rothe and Radu Timofte and Luc Van Gool},

  title = {Deep expectation of real and apparent age from a single image without facial landmarks},

  journal = {International Journal of Computer Vision},

  volume={126},

  number={2-4},

  pages={144--157},

  year={2018},

  publisher={Springer}

}


@InProceedings

{Rothe-ICCVW-2015,

  author = {Rasmus Rothe and Radu Timofte and Luc Van Gool},

  title = {DEX: Deep EXpectation of apparent age from a single image},

  booktitle = {IEEE International Conference on Computer Vision Workshops (ICCVW)},

  year = {2015},

  month = {December},

}

# <h1><b><font color='ORANGE'>GENDER CLASSIFIER</h1>

## **LIBRAIRES IMPORTATIONS**

In [ ]:
#Importing the libraries
import random
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model, layers
from tensorflow.keras.layers import Conv2D, SeparableConv2D, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

## **DATA IMPORTATION & DATA PRE-PROCESSING**

In [ ]:
############################################################################################################

#WE DEFINE THE CONSTANTS
META_DATA_PATH = 'h:/Desktop/Deep Learning/cs577-f22-bazz-mouhammad/project/data/imdb_meta/imdb/imdb.mat' #Path to the meta data
IMAGES_PATH = 'h:/Desktop/Deep Learning/cs577-f22-bazz-mouhammad/project/data/imdb_crop/' #Path to the images
IMG_SIZE = 64 #Size of the images

TEST_RATIO = 0.2 #Ratio of the test set
VAL_RATIO = 0.2 #Ratio of the validation set
RANDOM_STATE = 42 #Random state for the train_test_split function

NUMBER_IMDB_IMAGES = 100000 #Number of images in the imdb dataset


############################################################################################################
############################################################################################################

# ------------------------------ #
#SPECIFIC FOR OUR PROBLEM OF DATA IMPORTATION
# ------------------------------ #

PROBLEM_WITH_DATA_IMPORTATION = False

if PROBLEM_WITH_DATA_IMPORTATION:
    #We define only the folders we can use
    PATH_BEGINNING = [str(i)+str(j) for i in range(2) for j in range(10)]
    PATH_BEGINNING.append('20')
    PATH_BEGINNING.append('21')
    PATH_BEGINNING.append('22')
    PATH_BEGINNING.append('23')
    PATH_BEGINNING.append('24')
    PATH_BEGINNING.append('25')
    PATH_BEGINNING.append('26')
    PATH_BEGINNING.append('27')
    PATH_BEGINNING.append('28')

    #We define again the NUMBER_IMDB_IMAGES because we can only use a subset of the dataset
    NUMBER_IMDB_IMAGES = 130000
else:
    PATH_BEGINNING = [str(i)+str(j) for i in range(10) for j in range(10)]

# ------------------------------ #
# ------------------------------ #

#### *Importing the Meta Data*

In [ ]:
#We want to read the metadata file that is in .mat (MatLab) in order to know the labels of the images

#We define all the names of the different categories of the metadata of the images
CATEGORIES = ["date_of_birth", "photo_taken", "full_path", "gender", "name", "face_location", "face_score", "second_face_score", "celeb_names", "celeb_id"]
GENDER = ["FEMALE", "MALE"]

#We read the metadata file
mat = scipy.io.loadmat(META_DATA_PATH)

#We get only the part that we care about
meta_data = mat['imdb'][0,0]

#We will collect only the full_path (id) and the gender (label) of the images
#We create a list of tuples (id, label)
data = []
i = 0
#We iterate over all the images
for i in range(NUMBER_IMDB_IMAGES):
    #We only take the images that are in the folders we can use
    if (meta_data[2][0][i][0][0]+meta_data[2][0][i][0][1]) in PATH_BEGINNING:
      data.append((meta_data[2][0][i][0], meta_data[3][0][i]))



#ATTENTION: We do not have necessarily NUMBER_IMDB_IMAGES images because some of them are not in the folders we can use
print("Number of images in the dataset: ", len(data))



#### *Importing the Images using the meta data*

In [ ]:
#Importing the images (ATTENTION: This takes a while)

#We create a list of images
images = []
#We create a list of labels
labels = []
#We initialize the number of images we have seen
number_images = 0

#We iterate over all the images
for path, label in data:

    if number_images % 10000 == 0:
        print("Number of images we have seen: ", number_images)

    #we will select only the images that have a non corrupted label (0 or 1)
    if (label == 0) or (label == 1):
        try:
            #We read the image
            image = cv2.imread(IMAGES_PATH+path)
            #We resize the image
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            #We add the image to the list of images
            images.append(image)    
            #We add the label to the list of labels
            labels.append(label)
        except:
            pass

    #We increment the number of images we have seen
    number_images += 1


#### **Data Visualization**

In [ ]:
#VISUALIZATION OF THE DATA

GENDER = ["FEMALE", "MALE"]

#We have to visualize the data in order to see if the data is balanced or not
# - Visualize the data
# - Plot the Gender repartition

# - Visualize the data
#Plot one image of each gender
#fig, ax = plt.subplots(1, 2, figsize=(10, 5))
#for i in range(2):
#    ax[i].imshow(images[labels.index(i)].reshape(IMG_SIZE, IMG_SIZE, 3))
#    ax[i].set_title(GENDER[i])
    #We remove the ticks
#    ax[i].set_xticks([])
#    ax[i].set_yticks([])
#plt.show()

# - Plot the Gender repartition
#We plot the repartition 
#We create a list of the number of images for each label
number_images = [labels.count(0), labels.count(1)]
#We create a list of the percentage of images for each label
percentage_images = [labels.count(0)/len(labels), labels.count(1)/len(labels)]
#We create a figure
fig, ax = plt.subplots()
#We create a bar plot
ax.bar(GENDER, number_images)
#We add the percentage of images for each label
for i in range(2):
    ax.text(GENDER[i], number_images[i], str(round(percentage_images[i]*100, 2))+'%', ha='center', va='bottom')
#We add the title
ax.set_title('Repartition of the labels')
#We add the x label
ax.set_xlabel('Labels')
#We add the y label
ax.set_ylabel('Number of images')
#We show the plot
plt.show()

#We display the number of images we have in total
print("Number of images in the dataset: ", len(images))

#### **Balance the class repartition** (optional pre-processing)

In [ ]:
#BALANCING THE DATA (OPTIONAL) #WILL DELETE SOME IMAGES FROM A SPECIFIC CLASS

#We get the number of images for each label
number_images = [labels.count(0), labels.count(1)] 
#We get the index of the label with the most images
index_max = np.argmax(number_images) 


# - We balance the data by removing images of the label with the most images

#We get the indexes of the images of the class with the most images
indexes = [j for j, x in enumerate(labels) if x == index_max]
#We shuffle the indexes
random.shuffle(indexes)
#We truncate the list of images and labels
images_trunc = [images[j] for j in indexes[:number_images[1-index_max]]]
labels_trunc = [labels[j] for j in indexes[:number_images[1-index_max]]]


# - We balance the data by duplicating images of the label with the least images

#We get the indexes of the images of the class with the least images
indexes = [j for j, x in enumerate(labels) if x == (1-index_max)]
#We add the images of the class with the least images
images_trunc += [images[j] for j in indexes]
labels_trunc += [labels[j] for j in indexes]



#We shuffle the images and labels
c = list(zip(images_trunc, labels_trunc))
random.shuffle(c)
images_trunc, labels_trunc = zip(*c)


# - Plot the Gender repartition
#We create a list of the number of images for each label
number_images = [labels_trunc.count(0), labels_trunc.count(1)]
#We create a list of the percentage of images for each label
percentage_images = [labels_trunc.count(0)/len(labels_trunc), labels_trunc.count(1)/len(labels_trunc)]
#We create a figure
fig, ax = plt.subplots()
#We create a bar plot
ax.bar(GENDER, number_images)
#We add the percentage of images for each label
for i in range(2):
    ax.text(GENDER[i], number_images[i], str(round(percentage_images[i]*100, 2))+'%', ha='center', va='bottom')
#We add the title
ax.set_title('Repartition of the labels')
#We add the x label
ax.set_xlabel('Labels')
#We add the y label
ax.set_ylabel('Number of images')
#We show the plot
plt.show()


#We define the images and labels as images_trunc and labels_trunc
images = images_trunc
labels = labels_trunc

#We display the number of images we have in total after balancing the data
print("Number of images in the dataset: ", len(images))

#### **PRE-PROCESSING**

In [ ]:
#DATA PREPROCESSING

#We convert the list of images to a numpy array
images = np.array(images)
#We convert the list of labels to a numpy array
labels = np.array(labels)

#We split the data into train, test and validation
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=TEST_RATIO, random_state=RANDOM_STATE)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=VAL_RATIO, random_state=RANDOM_STATE)

#We normalize the data
x_train = x_train/255
x_test = x_test/255
x_val = x_val/255

#We one-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)


## **MODEL ARCHITECTURE**

In [ ]:
############################################################################################################

#WE DEFINE THE CONSTANTS
REGULARIZATION = 0.01 #The regularization parameter
LEARNING_RATE = 0.001 #The learning rate
IMG_SIZE = 64 #The size of the images
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3) #The shape of the images
OUTPUT_SHAPE = 2 #The shape of the output (number of classes)

############################################################################################################

# ------------------------------ #
# ------------------------------ #
# IMPORTANT:

#YOU HAVE TO CHOOSE THE MODEL THAT YOU WANT TO USE BY EXECUTING ONE CELL AMONG THE NEXT 2 FOLLOWING CELLS

# ------------------------------ #

### <font color='SKY BLUE'>**FIRST MODEL - RESIDUAL MODEL**

In [ ]:
#BUILDING THE MODEL

#We define the input
input = Input(shape=IMG_SHAPE)

#We add twice: one convolutional layer with Batch Normalization
x = Conv2D(9, (3, 3), strides=(1, 1), kernel_regularizer=tf.keras.regularizers.l2(REGULARIZATION), use_bias=False)(input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(9, (3, 3), strides=(1, 1), kernel_regularizer=tf.keras.regularizers.l2(REGULARIZATION), use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

#We will repeat the following 4 times:
#-- Then we will do in parallel: one convolutional layer with Batch Normalization
#-- and twice: one depth-wise seperable convolutional layer with Batch Normalization followed by a max pooling layer
#-- then we will add the two branches

#First repetition
#The convolutional layer with Batch Normalization
residual_1 = Conv2D(18, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual_1 = BatchNormalization()(residual_1)
#The depth-wise seperable convolutional layer with Batch Normalization followed by a max pooling layer
x1 = SeparableConv2D(18, (3, 3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(REGULARIZATION), use_bias=False)(x)
x1 = BatchNormalization()(x1)
x1 = Activation('relu')(x1)
x1 = SeparableConv2D(18, (3, 3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(REGULARIZATION), use_bias=False)(x1)
x1 = BatchNormalization()(x1)
x1 = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x1)
#We add the two branches
x = layers.add([x1, residual_1])

#Second repetition
#The convolutional layer with Batch Normalization
residual_2 = Conv2D(36, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual_2 = BatchNormalization()(residual_2)
#The depth-wise seperable convolutional layer with Batch Normalization followed by a max pooling layer
x2 = SeparableConv2D(36, (3, 3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(REGULARIZATION), use_bias=False)(x)
x2 = BatchNormalization()(x2)
x2 = Activation('relu')(x2)
x2 = SeparableConv2D(36, (3, 3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(REGULARIZATION), use_bias=False)(x2)
x2 = BatchNormalization()(x2)
x2 = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x2)
#We add the two branches
x = layers.add([x2, residual_2])

#Third repetition
#The convolutional layer with Batch Normalization
residual_3 = Conv2D(72, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual_3 = BatchNormalization()(residual_3)
#The depth-wise seperable convolutional layer with Batch Normalization followed by a max pooling layer
x3 = SeparableConv2D(72, (3, 3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(REGULARIZATION), use_bias=False)(x)
x3 = BatchNormalization()(x3)
x3 = Activation('relu')(x3)
x3 = SeparableConv2D(72, (3, 3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(REGULARIZATION), use_bias=False)(x3)
x3 = BatchNormalization()(x3)
x3 = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x3)
#We add the two branches
x = layers.add([x3, residual_3])


#Fourth repetition
#The convolutional layer with Batch Normalization
residual_4 = Conv2D(144, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual_4 = BatchNormalization()(residual_4)
#The depth-wise seperable convolutional layer with Batch Normalization followed by a max pooling layer
x4 = SeparableConv2D(144, (3, 3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(REGULARIZATION), use_bias=False)(x)
x4 = BatchNormalization()(x4)
x4 = Activation('relu')(x4)
x4 = SeparableConv2D(144, (3, 3), padding='same', kernel_regularizer=tf.keras.regularizers.l2(REGULARIZATION), use_bias=False)(x4)
x4 = BatchNormalization()(x4)
x4 = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x4)
#We add the two branches
x = layers.add([x4, residual_4])


#Now we add a convolutional layer then a global average pooling layer and then a softmax layer
x = Conv2D(OUTPUT_SHAPE, (3, 3))(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax')(x)

#We define the model
model = Model(input, output)

model.summary()


### <font color='SKY BLUE'> **SECOND MODEL - BASIC CNN MODEL**

In [ ]:
#BUILDING THE MODEL
#We will use the Sequential API to build the model

basic_model = Sequential()

# Layer 1
basic_model.add(Conv2D(16, kernel_size=(7, 7), padding='same', input_shape=IMG_SHAPE))
basic_model.add(BatchNormalization())
basic_model.add(Activation('relu'))
basic_model.add(Conv2D(16, kernel_size=(7, 7), padding='same'))
basic_model.add(BatchNormalization())
basic_model.add(Activation('relu'))
basic_model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
basic_model.add(Dropout(.5))

# Layer 2
basic_model.add(Conv2D(32, kernel_size=(5, 5), padding='same'))
basic_model.add(BatchNormalization())
basic_model.add(Conv2D(32, kernel_size=(5, 5), padding='same'))
basic_model.add(BatchNormalization())
basic_model.add(Activation('relu'))
basic_model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
basic_model.add(Dropout(.5))

# Layer 3
basic_model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
basic_model.add(BatchNormalization())
basic_model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
basic_model.add(BatchNormalization())
basic_model.add(Activation('relu'))
basic_model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
basic_model.add(Dropout(.5))

# Layer 4
basic_model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
basic_model.add(BatchNormalization())
basic_model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
basic_model.add(BatchNormalization())
basic_model.add(Activation('relu'))
basic_model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
basic_model.add(Dropout(.5))

# Output
basic_model.add(Conv2D(256, kernel_size=(3, 3), padding='same'))
basic_model.add(BatchNormalization())
basic_model.add(Conv2D(OUTPUT_SHAPE, kernel_size=(3, 3), padding='same'))
basic_model.add(GlobalAveragePooling2D())
basic_model.add(Activation('softmax'))

#We render basic_model into model
model = basic_model


## **COMPILE MODEL**

In [ ]:
#We compile the model

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])

## **MODEL TRAINING**

In [ ]:
############################################################################################################

#WE DEFINE THE CONSTANTS
EPOCHS = 10 #Number of epochs
BATCH_SIZE = 32 #Batch size
WITH_DATA_AUGMENTATION = False #Use of data augmentation or not

# ------------------------------ #
#SPECIFIC FOR THE DATA AUGMENTATION
# ------------------------------ #

#We define the data augmentation
data_augmentation = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest')

# ------------------------------ #
# ------------------------------ #


In [ ]:
#TRAINING THE MODEL

#We train the model
if WITH_DATA_AUGMENTATION:
    train_generator = data_augmentation.flow(x_train, y_train, batch_size=BATCH_SIZE)
    steps_per_epoch = train_generator.n // train_generator.batch_size
    history = model.fit(train_generator, epochs=EPOCHS, steps_per_epoch=steps_per_epoch, validation_data=(x_val, y_val))
else:
    history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(x_val, y_val))


In [ ]:
#PLOTTING THE RESULTS

#We plot the results
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## **MODEL EVALUATION**

In [ ]:
############################################################################################################

#WE DEFINE THE CONSTANTS
IDEAL_EPOCH = 1 #We choose the ideal epoch (thanks to the plot above)

############################################################################################################

In [ ]:
#EVALUATING THE MODEL

# - First we have to train again the model with the whole training set (train + validation)

x_train_val = np.concatenate((x_train, x_val), axis=0)
y_train_val = np.concatenate((y_train, y_val), axis=0)

if WITH_DATA_AUGMENTATION:
    train_generator = data_augmentation.flow(x_train_val, y_train_val, batch_size=BATCH_SIZE)
    steps_per_epoch = train_generator.n // train_generator.batch_size
    history = model.fit(train_generator, epochs=IDEAL_EPOCH, steps_per_epoch=steps_per_epoch)
else:
    history = model.fit(x_train_val, y_train_val, batch_size=BATCH_SIZE, epochs=IDEAL_EPOCH, verbose=0)


# - Then we evaluate the model on the test set

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
#CONFUSION MATRIX

GENDER = ["Woman", "Man"]

#We compute the predictions
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

#We store the true labels
y_true = np.argmax(y_test, axis=1)

#We compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

#We plot the confusion matrix
df_cm = pd.DataFrame(cm, index = [i for i in GENDER],
                    columns = [i for i in GENDER])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True, fmt='g')
plt.title('Confusion matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()

## **WE SAVE THE MODEL**

In [ ]:
#We save the model

############################################################################################################
#WE DEFINE THE CONSTANTS
name = 'model' #Name of the model

############################################################################################################



data_augmented = 'data_augmented' if WITH_DATA_AUGMENTATION else ''

model.save(name + '_' + data_augmented + '.h5')

## **WE LOAD A MODEL**

In [ ]:
#We load the model

############################################################################################################
#WE DEFINE THE CONSTANTS
file_name = 'model' #Name of the file

############################################################################################################

model = load_model(file_name + '.h5')
